# Ireland Electricity Demand Forecasting
## Hybrid Interpretable Machine Learning Model

**Author:** Prathima Project  
**Data Source:** Open Power System Data (OPSD)  

---

### Research Objectives
1. Develop a hybrid ML model combining XGBoost and LSTM
2. Implement SHAP-based explainability
3. Evaluate robustness under extreme demand conditions
4. Create a reproducible benchmark using open data

In [ ]:
# Setup and Imports
import sys
import warnings
from pathlib import Path

# Add project root to path
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

print("Setup complete!")

## 1. Data Loading

We use the Open Power System Data (OPSD) which provides transparent, reproducible electricity data for Ireland.

In [ ]:
from config.config import DATA_DIR, PLOTS_DIR, RESULTS_DIR
from src.data_loader import load_and_prepare_data

# Load data (downloads if not present)
df = load_and_prepare_data(force_download=False)

print(f"\nData Shape: {df.shape}")
print(f"Date Range: {df.index.min()} to {df.index.max()}")
df.head()

In [ ]:
# Basic Statistics
print("Demand Statistics:")
df['demand'].describe()

## 2. Exploratory Data Analysis (EDA)

In [ ]:
# Time Series Overview
fig, ax = plt.subplots(figsize=(16, 6))
ax.plot(df.index, df['demand'], linewidth=0.5, alpha=0.7)
ax.set_title('Ireland Electricity Demand Over Time', fontsize=14)
ax.set_xlabel('Date')
ax.set_ylabel('Demand (MW)')
plt.tight_layout()
plt.show()

In [ ]:
# Daily Pattern
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Hourly pattern
hourly_mean = df.groupby(df.index.hour)['demand'].mean()
hourly_std = df.groupby(df.index.hour)['demand'].std()

axes[0].fill_between(hourly_mean.index, hourly_mean - hourly_std, 
                      hourly_mean + hourly_std, alpha=0.3)
axes[0].plot(hourly_mean.index, hourly_mean, linewidth=2, marker='o')
axes[0].set_title('Average Daily Demand Pattern', fontsize=12)
axes[0].set_xlabel('Hour of Day')
axes[0].set_ylabel('Demand (MW)')
axes[0].set_xticks(range(0, 24, 2))

# Weekly pattern
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
daily_mean = df.groupby(df.index.dayofweek)['demand'].mean()
daily_std = df.groupby(df.index.dayofweek)['demand'].std()

axes[1].bar(range(7), daily_mean, yerr=daily_std, capsize=3, alpha=0.7)
axes[1].set_title('Average Weekly Demand Pattern', fontsize=12)
axes[1].set_xlabel('Day of Week')
axes[1].set_ylabel('Demand (MW)')
axes[1].set_xticks(range(7))
axes[1].set_xticklabels(days)

plt.tight_layout()
plt.show()

In [ ]:
# Heatmap: Hour vs Day of Week
pivot = df.pivot_table(values='demand', 
                        index=df.index.hour, 
                        columns=df.index.dayofweek, 
                        aggfunc='mean')
pivot.columns = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

plt.figure(figsize=(10, 8))
sns.heatmap(pivot, cmap='YlOrRd', annot=True, fmt='.0f', cbar_kws={'label': 'MW'})
plt.title('Average Hourly Demand by Day of Week', fontsize=14)
plt.xlabel('Day of Week')
plt.ylabel('Hour of Day')
plt.tight_layout()
plt.show()

## 3. Feature Engineering

In [ ]:
from src.feature_engineering import engineer_features, get_feature_columns

# Apply feature engineering
df_featured = engineer_features(df)

print(f"\nOriginal columns: {len(df.columns)}")
print(f"After feature engineering: {len(df_featured.columns)}")
print(f"\nNew features created: {len(df_featured.columns) - len(df.columns)}")

In [ ]:
# View feature columns
feature_cols = get_feature_columns(df_featured)
print(f"Feature columns ({len(feature_cols)}):")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i}. {col}")

## 4. Data Preparation

In [ ]:
from src.preprocessing import split_data

# Split data into train, validation, test
train_df, val_df, test_df = split_data(df_featured)

# Prepare X and y
X_train = train_df[feature_cols]
y_train = train_df['demand'].values

X_val = val_df[feature_cols]
y_val = val_df['demand'].values

X_test = test_df[feature_cols]
y_test = test_df['demand'].values

print(f"\nTraining set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Test set: {X_test.shape}")

## 5. Model Training

In [ ]:
from src.models import RandomForestModel, XGBoostModel, HybridModel

models = {}

In [ ]:
# Train Random Forest
print("Training Random Forest...")
rf_model = RandomForestModel()
rf_model.fit(X_train, y_train)
models['Random Forest'] = rf_model

In [ ]:
# Train XGBoost
print("Training XGBoost...")
xgb_model = XGBoostModel()
xgb_model.fit(X_train, y_train, X_val, y_val)
models['XGBoost'] = xgb_model

In [ ]:
# Train Hybrid Model (XGBoost + LSTM)
print("Training Hybrid Model...")
try:
    hybrid_model = HybridModel()
    hybrid_model.fit(X_train, y_train, X_val, y_val, feature_cols)
    models['Hybrid'] = hybrid_model
except Exception as e:
    print(f"Hybrid model training failed: {e}")
    print("Continuing with tree-based models...")

## 6. Model Evaluation

In [ ]:
from src.evaluation import evaluate_models, calculate_metrics

# Evaluate all models
results_df = evaluate_models(models, X_test, y_test)
print("\nModel Comparison:")
results_df

In [ ]:
# Visualize model comparison
from src.visualization import plot_model_comparison

plot_model_comparison(results_df)

In [ ]:
# Best model predictions
best_model_name = results_df.iloc[0]['model']
best_model = models[best_model_name]
y_pred = best_model.predict(X_test)

# Plot predictions
from src.visualization import plot_predictions

plot_predictions(y_test, y_pred, dates=test_df.index, model_name=best_model_name)

## 7. Feature Importance

In [ ]:
# Get feature importance from XGBoost
importance = xgb_model.get_feature_importance(top_n=20)
print("Top 20 Important Features:")
importance

In [ ]:
from src.visualization import plot_feature_importance

plot_feature_importance(importance, title='XGBoost Feature Importance')

## 8. SHAP Explainability Analysis

In [ ]:
from src.explainability import compute_shap_values, plot_shap_summary, plot_shap_bar

# Compute SHAP values
shap_values, explainer, X_sample = compute_shap_values(xgb_model, X_test, feature_cols)

In [ ]:
# SHAP Summary Plot
plot_shap_summary(shap_values, X_sample, feature_cols)

In [ ]:
# SHAP Bar Plot
shap_importance = plot_shap_bar(shap_values, feature_cols)

In [ ]:
# SHAP Dependence Plot for top features
from src.explainability import plot_shap_dependence

top_feature = shap_importance.iloc[-1]['feature']  # Most important
plot_shap_dependence(shap_values, X_sample, top_feature)

## 9. Extreme Event Analysis

In [ ]:
from src.evaluation import evaluate_extreme_events

# Analyze performance during extreme events
extreme_results = evaluate_extreme_events(best_model, X_test, y_test, test_df)

In [ ]:
# Compare performance across conditions
conditions = ['normal_conditions', 'high_demand', 'low_demand']
metrics_comparison = []

for condition in conditions:
    if condition in extreme_results:
        m = extreme_results[condition]
        metrics_comparison.append({
            'Condition': condition.replace('_', ' ').title(),
            'MAE': m['MAE'],
            'RMSE': m['RMSE'],
            'MAPE': m['MAPE']
        })

pd.DataFrame(metrics_comparison)

## 10. Hourly Performance Analysis

In [ ]:
from src.evaluation import evaluate_by_time_period

# Analyze by time period
time_analysis = evaluate_by_time_period(best_model, X_test, y_test, test_df)

if 'hourly' in time_analysis:
    print("Hourly Performance:")
    display(time_analysis['hourly'])

In [ ]:
# Plot hourly performance
from src.visualization import plot_hourly_performance

if 'hourly' in time_analysis:
    plot_hourly_performance(time_analysis['hourly'], metric='MAPE')

## 11. Conclusions

### Key Findings:

1. **Model Performance**: The hybrid/XGBoost model achieves competitive accuracy for hourly forecasting

2. **Important Features**: 
   - Lag features (previous hour, previous day) are most important
   - Temporal features (hour, day of week) capture demand patterns
   - Rolling statistics help smooth predictions

3. **Explainability**: SHAP analysis reveals:
   - Clear impact of time-of-day on predictions
   - Weekend/weekday effects on demand
   - Seasonal variations in importance

4. **Robustness**: Model performance varies under extreme conditions

### Contributions:
- Reproducible benchmark using open OPSD data
- Hybrid architecture combining XGBoost and LSTM
- SHAP-based interpretability for transparency

In [ ]:
# Final Summary
print("="*60)
print("FINAL SUMMARY")
print("="*60)
print(f"\nBest Model: {best_model_name}")
print(f"Test RMSE: {results_df.iloc[0]['RMSE']:.2f} MW")
print(f"Test MAPE: {results_df.iloc[0]['MAPE']:.2f}%")
print(f"Test R²: {results_df.iloc[0]['R2']:.4f}")